# Simulador de daltonismo

Bruna Kimura e Elisa Malzoni

## Simulador de protanopia

Este prieiro projeto consiste em simular em imagens como um daltônico (protanopia e deuteranopia) as veriam. Para tanto foi utilizado como base o artigo "Digital Video Colourmaps for Checkinh the Legibility of Displays by Dichromats" de Viénot, Brettel e Mollon.

### Passo 1:
Tendo os valores em RGB, o primeiro passo é ajustar o fluxo RGB

### Passo 2:
Transformar o RGB em sinais LMS

### Passo 3:
